# 04 — Targets, Feature und Matrizen erstellen

## Ziel
Dieses Notebook erstellt die Zielvariablen (Targets) und die Feature-Matrizen für die Modellierung. Es speichert die vorbereiteten Daten und Modell-Pipelines für die spätere Verwendung.

Am Ende erzeugen wir:
- Zielvariablen (Targets) für die Modellierung
- Feature-Matrizen für die Modellierung


In [1]:
import sys, importlib

# reload in der richtigen Reihenfolge (dependencies zuerst, dann consumer)
for m in [
    "utils.data.parsing",
    "utils.features.numeric_transform",
    "utils.data.catalog",
    "utils.datasets_modeling.track_dataset",
    "utils.datasets_modeling.album_dataset",
    "utils.features.engineering",
    "utils.features.multihot",
    "utils.targets.config",
    "utils.targets.builder",
    "utils.modeling.feature_schema",
    "utils.modeling.artifact_store",
    "utils.config.settings",
    "utils.core.paths",
    "utils.targets.config",
    "utils.targets.builder",
    "utils.modeling.feature_schema",
    "utils.modeling.artifact_store",
    "utils.config.settings",
    "utils.modeling.feature_schema",
    "utils.targets.hit",
]:
    if m in sys.modules:
        importlib.reload(sys.modules[m])

## Imports und Setup


In [2]:
import pandas as pd
import importlib

from utils.features.multihot import top_k_list_counts,genres_to_multihot
from utils.data.catalog import TableCatalog, TableCatalogConfig
from utils.targets.config import (
    CohortConfig, HitConfig, MoodConfig, MoodTagDef, ArtistTrajectoryConfig
)
from utils.modeling.feature_schema import FeatureSchemaBuilder, FeatureSchemaConfig
from utils.targets.builder import TargetsBuilder, TargetsBuilderConfig
from utils.config.settings import (
TOP_K_GENRES
)
from utils.modeling.artifact_store import ArtifactStore, ArtifactStoreConfig
import utils.core.paths as paths

importlib.reload(paths)

SAMPLE_NAME = paths.load_sample_name()
PATHS = paths.make_paths(SAMPLE_NAME)
paths.ensure_dirs(PATHS)



## Laden von Daten

In [3]:
TABLES = [
    "track_dataset", "album_dataset", "artist_dataset"
]

catalog = TableCatalog(TableCatalogConfig(PATHS.modeling_dir, TABLES))
data = catalog.load(strict=False)
track_df = data["track_dataset"]
album_df = data["album_dataset"]
artist_df = data["artist_dataset"]


dur_col = "duration" if "duration" in track_df.columns else ("duration_ms" if "duration_ms" in track_df.columns else None)

## Zielvariablen (Targets)

Für die Modellierung werden mehrere **Zielvariablen (Targets)** definiert, die unterschiedliche
Lernaufgaben abdecken (Regression, Klassifikation, Ranking, Multi-Label).

Die Targets werden **ausschließlich auf Track-Ebene** konstruiert und sind zeit- und kohortenrobust.

---

### (A) Success Percentile within Cohort
**Ranking-Target / Top-K-Aufgabe**

Misst die **relative Erfolgsposition eines Tracks** innerhalb seiner zeitlichen Release-Kohorte
(z. B. Jahr oder Jahr-Monat).

- Kohortenbasierter Perzentil-Rank
- Robust gegenüber Katalog- und Zeitverschiebungen
- Geeignet für Ranking- und Recommendation-Modelle

---

### (B) Success Residual within Cohort
**Overperformance-Target (Regression)**

Erfasst, ob ein Track **über oder unter dem erwarteten Kohorten-Durchschnitt** performt.

- Differenz zur mittleren Kohorten-Popularität
- Kontrolliert für Release-Zeitpunkt
- Nützlich zur Erkennung von „Sleeper Hits“

---

### (C) Hit-Vorhersage
**Binäres Klassifikationsziel**

Kennzeichnet Tracks als *Hit* oder *Non-Hit* basierend auf **jahresrelativen Popularitäts-Schwellenwerten**.

- Perzentilbasierte Definition (z. B. Top-10 % pro Jahr)
- Stabil gegenüber langfristigen Popularitäts-Trends
- Optionales Rate-Clipping zur Sicherstellung ausreichender Positivrate

---

### (D) Mood-Tags
**Multi-Label-Target (schwache Supervision)**

Leitet **semantische Stimmungs-Labels** aus Audio-Features ab.

- Schwellenwerte über Quantile (z. B. Energy, Valence, Danceability)
- Keine manuelle Annotation nötig
- Eignet sich für Representation Learning & Tag-Prediction

---

### (E) Artist Trajectory Target
**Zeitverschobenes Wachstums-Target**

Beschreibt die **zukünftige Popularitätsentwicklung eines Künstlers** auf Basis aggregierter Track-Performance.

- Rolling-Fenster (Vergangenheit vs. Zukunft)
- Regression (Popularity-Δ) oder Klassifikation (Breakout-Indikator)
- Unterstützt Karriere- und Trendanalysen

---

**Hinweis:**
Alle Targets werden aus `track_df` abgeleitet. Artist- und Album-Informationen werden ausschließlich
über Aggregationen eingebracht, wodurch eine einheitliche Ereignis-Basis erhalten bleibt.


In [4]:
MOOD_CFG = MoodConfig(tags=[
    MoodTagDef("energetic", "energy", 0.75, "gt"),
    MoodTagDef("danceable", "danceability", 0.75, "gt"),
    MoodTagDef("acoustic", "acousticness", 0.75, "gt"),
    MoodTagDef("instrumental", "instrumentalness", 0.75, "gt"),
    MoodTagDef("happy", "valence", 0.75, "gt"),
    MoodTagDef("sad", "valence", 0.25, "lt"),
    MoodTagDef("chill", "energy", 0.25, "lt"),
])

cfg = TargetsBuilderConfig(
    cohort=CohortConfig(),
    hit=HitConfig(hit_percentile=0.90, desired_rate=0.10, min_tracks_per_year=200),
    mood=MOOD_CFG,
    traj=ArtistTrajectoryConfig(past_m=6, future_m=6, min_past_tracks=5, breakout_q=0.90),
)

tb = TargetsBuilder(cfg)
out = tb.build(track_df)

track_df = out["track_df_with_targets"]
artist_panel = out["artist_panel"]
y_hit = out["y_hit"]
Y_mood = out["Y_mood"]
y_artist_growth = out["y_artist_growth"].astype("float32")
y_artist_breakout = out["y_artist_breakout"].astype("int8")
y_success_pct = out["y_success_pct"]
y_success_residual = out["y_success_residual"]

## Genre Multi-Hot (Top-K) für Track / Album / Artist

Genres liegen als **Listen** vor
(z. B. `track_genres = [genre_id1, genre_id2, ...]`).

Da die meisten ML-Modelle **feste numerische Vektoren** benötigen, gehen wir wie folgt vor:

1. **Top-K Genres bestimmen**
   – die K häufigsten Genres aus `track_df`

2. **Multi-Hot Encoding erzeugen**
   – für jedes Top-K-Genre eine 0/1-Spalte
   – 1 = Genre vorhanden, 0 = Genre nicht vorhanden

### Warum Top-K?

- Der komplette Genre-Raum ist sehr groß
- Top-K hält die Feature-Dimension **überschaubar**
- Vermeidet extrem sparse Feature-Matrizen
- Seltene Genres werden implizit als **„Other“** behandelt
  (alle 0 in den Top-K-Spalten)



In [5]:
# 1) Top-K Genres aus Track-Level definieren (einheitliches Vokabular)
top_genres = (
    top_k_list_counts(track_df["track_genres"], top_k=TOP_K_GENRES)
    if "track_genres" in track_df.columns else []
)

# 2) Multi-hot pro Tabelle (Key-stabil)
track_genre_mh = (
    genres_to_multihot(track_df, "track_genres", top_genres, prefix="track_")
    if top_genres else pd.DataFrame(index=track_df.index)
)

album_genre_mh = (
    genres_to_multihot(album_df.set_index("album_id"), "album_genres", top_genres, prefix="album_")
    if (top_genres and "album_genres" in album_df.columns and "album_id" in album_df.columns)
    else pd.DataFrame(index=album_df["album_id"] if "album_id" in album_df.columns else album_df.index)
)

artist_genre_mh = (
    genres_to_multihot(artist_df.set_index("artist_id"), "artist_genres", top_genres, prefix="artist_")
    if (top_genres and "artist_genres" in artist_df.columns and "artist_id" in artist_df.columns)
    else pd.DataFrame(index=artist_df["artist_id"] if "artist_id" in artist_df.columns else artist_df.index)
)

print("Genre multi-hot shapes:", track_genre_mh.shape, album_genre_mh.shape, artist_genre_mh.shape)

Genre multi-hot shapes: (300000, 0) (195981, 0) (187090, 0)


## Feature Selection & Leakage Guards (Task-spezifisch)

In diesem Abschnitt erzeugen wir die **finalen Feature-Matrizen (`X_*`)** für die Modellierung.
Ausgehend von den reichhaltigen, denormalisierten Tabellen (`track_df`, `album_df`, `artist_df`)
werden **aufgaben­spezifische und leakage-sichere Feature-Sets** definiert.

Statt einer einzigen globalen Feature-Matrix werden die Features **pro Modellierungsaufgabe**
gezielt ausgewählt.

### Ziele dieses Schritts
- Klare Trennung von **Rohdaten** und **Model-Inputs**
- Systematische Vermeidung von **Data Leakage**
- Reproduzierbares, konfigurierbares **Feature-Schema**
- Konsistente Feature-Auswahl für Training **und** Inference
- Einheitliche Genre-Repräsentation über **Top-K Multi-Hot Encoding**

---

### Grundprinzipien

#### 1. No-ID / No-URL Policy
- Technische Identifikatoren (`track_id`, `album_id`, `artist_id`)
  sowie URLs/URIs sind **grundsätzlich ausgeschlossen**.
- Diese Spalten führen sonst zu Memorisation und künstlich hoher Performance.

#### 2. Task-spezifische Leakage Guards
- Popularity und daraus abgeleitete Größen sind **Targets**
  (direkt oder indirekt bei Success / Hit / Ranking).
- Deshalb gelten unterschiedliche Regeln je nach Task:
  - **Hit / Success / Ranking**: keine Popularity- oder Reach-Proxies
  - **Mood**: rein audio-basierte + strukturelle Features
  - **Artist Trajectory**: ausschließlich vergangenheitsbasierte Aggregationen
- Reichweiten-Proxies (Followers, Artist-Popularity) sind **optional**
  und nur für explizite Experimente aktivierbar.

#### 3. Feature-Gruppen statt Einzelspalten
Features werden logisch gruppiert und nicht ad-hoc gewählt:
- strukturelle Metadaten (Track / Album)
- zeitliche Merkmale (Release-Jahr, Dekade, Flags)
- Audio-Features
- binäre Heuristiken / Flags
- Genre-Multi-Hot (Top-K)

Diese Struktur ermöglicht:
- saubere Erweiterungen
- konsistente Ablationen
- stabile Inference-Pipelines

---

### Ergebnis

- **`X_track_hit`** – Feature-Matrix für Hit-Klassifikation
- **`X_track_success`** – Feature-Matrix für Success-Regression / Ranking
- **`X_track_mood`** – Feature-Matrix für Multi-Label Mood Prediction
- **`X_album`** – Feature-Matrix für album-basierte Modelle
- **`X_artist`** – Feature-Matrix für artist-basierte Modelle
- Zusätzlich: **Schema- und Debug-Reports**, die dokumentieren,
  welche Spalten verwendet oder bewusst ausgeschlossen wurden.

Die Feature-Schemata werden in allen folgenden Notebooks
**unverändert wiederverwendet**, um Trainings- und Inference-Pipelines
konsistent und vergleichbar zu halten.

In [6]:
schema = FeatureSchemaBuilder(
    FeatureSchemaConfig(
        allow_text_features=True,
        allow_reach_proxies=False,
        allow_album_popularity_proxy=False,
        prefer_duration_col="duration_af",
    )
)

X_track_hit = schema.build_track_matrix(track_df, task="hit", track_genre_mh=track_genre_mh)["X_track"]
X_track_success = schema.build_track_matrix(track_df, task="success_pct", track_genre_mh=track_genre_mh)["X_track"]
X_track_mood = schema.build_track_matrix(track_df, task="mood", track_genre_mh=track_genre_mh)["X_track"]
X_album = schema.build_album_matrix(album_df, album_genre_mh=album_genre_mh)["X_album"]
X_artist = schema.build_artist_matrix(artist_df, artist_genre_mh=artist_genre_mh, allow_popularity=True)["X_artist"]

panel_feature_cols = [c for c in artist_panel.columns if c.startswith("past_")] + [
    "past_unique_tracks"
]
X_artist_panel = artist_panel[panel_feature_cols].copy()


## Speicher von Datasets

In [7]:
store = ArtifactStore(ArtifactStoreConfig(out_dir=PATHS.input_targets_path))

# datasets
store.save_df("track_df", track_df)
store.save_df("album_df", album_df)
store.save_df("artist_df", artist_df)
store.save_df("artist_panel", artist_panel)

# feature matrices
store.save_df("X_track_hit", X_track_hit)
store.save_df("X_track_success", X_track_success)
store.save_df("X_track_mood", X_track_mood)
store.save_df("X_album", X_album)
store.save_df("X_artist", X_artist)

# targets
store.save_series("y_hit", y_hit)
store.save_series("y_success_pct",y_success_pct)
store.save_series("y_success_residual", y_success_residual)
store.save_df("Y_mood", Y_mood)

store.save_series("y_artist_growth", y_artist_growth)
store.save_series("y_artist_breakout", y_artist_breakout)

# multi-hot
store.save_df("track_genre_mh", track_genre_mh)
store.save_df("album_genre_mh", album_genre_mh)
store.save_df("artist_genre_mh", artist_genre_mh)

store.finalize()

WindowsPath('C:/GitHub/uni-project-metrics-and-data/data/baseline_models_datasets/slice_000/_manifest.parquet')